In [54]:
import pandas as pd
import scipy.stats as st
from statsmodels.stats import proportion

In [11]:
df = pd.read_excel('AB_Test_Results (2) (1).xlsx')
df


C:\Users\MoHaX28reg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
...,...,...,...
9995,1981,control,0.0
9996,502,variant,0.0
9997,9214,variant,0.0
9998,7741,control,0.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [16]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [13]:
duplicateRows = df[df.duplicated(['USER_ID'])]
duplicateRows

,USER_ID,VARIANT_NAME,REVENUE
106,7704,control,0.0
195,8406,variant,0.0
224,7042,variant,0.0
283,4064,variant,0.0
302,4409,control,0.0
...,...,...,...
9992,9303,variant,0.0
9993,2400,variant,0.0
9994,3129,control,0.0
9995,1981,control,0.0


In [19]:
df_dup = df.drop_duplicates(subset = 'USER_ID', keep = 'first', inplace = False)
df_dup

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
...,...,...,...
9988,428,control,0.0
9991,8864,variant,0.0
9996,502,variant,0.0
9998,7741,control,0.0


Количество повторяющихся пользователей привышает 5%. Следовательно исключить их не можем.

In [47]:
df[df['REVENUE'] > 0].VARIANT_NAME.value_counts()

VARIANT_NAME
control    80
variant    72
Name: count, dtype: int64

In [22]:
df_a = df[df['VARIANT_NAME'] == 'variant']
df_b = df[df['VARIANT_NAME'] == 'control']


In [23]:
df_a.describe()

,USER_ID,REVENUE
count,5016.000000,5016.000000
mean,4972.776914,0.070070
std,2876.320625,1.314802
min,3.000000,0.000000
25%,2476.500000,0.000000
50%,4958.500000,0.000000
75%,7415.250000,0.000000
max,10000.000000,58.630000


In [42]:
all_a = df_a.REVENUE.value_counts().sum()
bay_a = df_a[df_a['REVENUE'] != 0].value_counts().sum()
all_a, bay_a

(5016, 72)

In [24]:
df_b.describe()

,USER_ID,REVENUE
count,4984.000000,4984.000000
mean,4989.436798,0.129013
std,2905.145109,3.007524
min,2.000000,0.000000
25%,2466.000000,0.000000
50%,4964.500000,0.000000
75%,7576.250000,0.000000
max,10000.000000,196.010000


In [43]:
all_b = df_b.REVENUE.value_counts().sum()
bay_b = df_b[df_b['REVENUE'] != 0].value_counts().sum()
all_b, bay_b

(4984, 80)

In [56]:
st.shapiro(df_a['REVENUE'])

C:\Users\MoHaX28reg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.027033090591430664, pvalue=0.0)

In [57]:
st.shapiro(df_b['REVENUE'])

ShapiroResult(statistic=0.018331527709960938, pvalue=0.0)

Распределение не нормальное. Используем критерий Манна-Уитни

In [58]:
st.mannwhitneyu(df_a['REVENUE'], df_b['REVENUE'])

MannwhitneyuResult(statistic=12478180.0, pvalue=0.47825247965294926)

pvalue=0.47, следовательно значемых различий между вариантами нет

In [59]:
df_a['REVENUE'].mean(), df_b['REVENUE'].mean()

(0.07006977671451356, 0.12901284109149277)

# Вариант 'control' приносит в среднем больше прибыли, следовательно выгоднее выбирать его